##### Copyright 2025 Google LLC.

In [ ]:
# @title Licenciado bajo la Licencia Apache, Versión 2.0 (la "Licencia");
# no puedes usar este archivo excepto en cumplimiento con la Licencia.
# Puedes obtener una copia de la Licencia en
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# A menos que sea requerido por la ley aplicable o acordado por escrito, el software
# distribuido bajo la Licencia se distribuye "TAL CUAL",
# SIN GARANTÍAS O CONDICIONES DE NINGÚN TIPO, ya sean expresas o implícitas.
# Consulta la Licencia para conocer el lenguaje específico que rige los permisos y
# limitaciones bajo la Licencia.

# Día 1 - Creación de Prompts

¡Bienvenido al curso de 5 días de Generative AI en Kaggle!

Este notebook te mostrará cómo comenzar con la API de Gemini y te guiará a través de algunos ejemplos de prompts y técnicas que también puedes leer en el documento técnico sobre Prompting. No necesitas leer el documento técnico para usar este notebook, pero los documentos te darán un contexto teórico y antecedentes para complementar este notebook interactivo.


## Antes de comenzar

En este notebook, comenzarás a explorar la creación de prompts utilizando el SDK de Python y AI Studio. Para inspirarte, podrías disfrutar explorando algunas aplicaciones que se han creado utilizando la familia de modelos Gemini. Aquí hay algunas que nos gustan, y creemos que a ti también te gustarán.

* [TextFX](https://textfx.withgoogle.com/) es un conjunto de herramientas impulsadas por IA para raperos, creado en colaboración con Lupe Fiasco,
* [SQL Talk](https://sql-talk-r5gdynozbq-uc.a.run.app/) muestra cómo puedes hablar directamente con una base de datos utilizando la API de Gemini,
* [NotebookLM](https://notebooklm.google/) utiliza modelos Gemini para construir tu propio asistente de investigación personal impulsado por IA.


## Para obtener ayuda

**Los problemas comunes están cubiertos en la [guía de preguntas frecuentes y solución de problemas](https://www.kaggle.com/code/markishere/day-0-troubleshooting-and-faqs).**

## ¡Novedades de Gemini 2.0!

Este material del curso se lanzó por primera vez en noviembre de 2024. El espacio de IA y LLM está avanzando increíblemente rápido, por lo que hemos realizado algunas actualizaciones para usar los últimos modelos y capacidades.

* Estos laboratorios de código se han actualizado para usar la familia de modelos Gemini 2.0.
* El SDK de Python se ha actualizado de `google-generativeai` al nuevo y unificado SDK [`google-genai`](https://pypi.org/project/google-genai).
  * Este nuevo SDK funciona tanto con la API de desarrolladores de Gemini como con Google Cloud Vertex AI, y cambiar entre ellos es [tan simple como cambiar algunos campos](https://pypi.org/project/google-genai/#:~:text=.Client%28%29-,API%20Selection,-By%20default%2C%20the).
* Se han agregado nuevas capacidades de modelo a los laboratorios de código relevantes, como el "modo de pensamiento" en este laboratorio.
* El Día 1 incluye un nuevo [laboratorio de evaluación](https://www.kaggle.com/code/markishere/day-1-evaluation-and-structured-output).

## Comenzar con los notebooks de Kaggle

Si es la primera vez que usas un notebook de Kaggle, ¡bienvenido! Puedes leer sobre cómo usar los notebooks de Kaggle [en la documentación](https://www.kaggle.com/docs/notebooks).

Primero, necesitarás verificar tu cuenta por teléfono en kaggle.com/settings.

![](https://storage.googleapis.com/kaggle-media/Images/5dgai_0.png)

Para ejecutar este notebook, así como los demás en este curso, necesitarás hacer una copia o bifurcar el notebook. Busca el botón `Copy and Edit` en la parte superior derecha y **haz clic en él** para hacer una copia privada editable del notebook. Debería verse como este:

![Botón Copy and Edit](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_1.png)

Tu copia ahora tendrá un botón ▶️ **Run** junto a cada celda de código que puedes presionar para ejecutar esa celda. Se espera que estos notebooks se ejecuten en orden de arriba hacia abajo, pero se te anima a agregar nuevas celdas, ejecutar tu propio código y explorar. Si te quedas atascado, puedes intentar la opción `Factory reset` en el menú `Run`, o volver al notebook original y hacer una nueva copia.

![Botón Run cell](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_2.png)

### ¿Problemas?

Si tienes algún problema, dirígete al [Discord de Kaggle](https://discord.com/invite/kaggle), encuentra el canal [`#5dgai-q-and-a`](https://discord.com/channels/1101210829807956100/1303438695143178251) y pide ayuda.

## Comenzar con la API de Gemini

Todos los ejercicios en este notebook utilizarán la [API de Gemini](https://ai.google.dev/gemini-api/) a través del [SDK de Python](https://pypi.org/project/google-genai/). Cada uno de estos prompts se puede acceder directamente en [Google AI Studio](https://aistudio.google.com/) también, por lo que si prefieres usar una interfaz web y omitir el código para esta actividad, busca el enlace <img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> AI Studio en cada prompt.

A continuación, necesitarás agregar tu clave API a tu Notebook de Kaggle como un Secreto de Usuario de Kaggle.

![](https://storage.googleapis.com/kaggle-media/Images/5dgai_1.png)
![](https://storage.googleapis.com/kaggle-media/Images/5dgai_2.png)
![](https://storage.googleapis.com/kaggle-media/Images/5dgai_3.png)
![](https://storage.googleapis.com/kaggle-media/Images/5dgai_4.png)

### Instalar el SDK

In [3]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"

Importa el SDK y algunos ayudantes para renderizar la salida.

In [4]:
from google import genai
from google.genai import types

from IPython.display import HTML, Markdown, display

Configura un ayudante de reintento. Esto te permite "Ejecutar todo" sin preocuparte por la cuota por minuto.

In [5]:
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

### Configura tu clave API

Para ejecutar la siguiente celda, tu clave API debe estar almacenada en un [secreto de Kaggle](https://www.kaggle.com/discussions/product-feedback/114053) llamado `GOOGLE_API_KEY`.

Si aún no tienes una clave API, puedes obtener una en [AI Studio](https://aistudio.google.com/app/apikey). Puedes encontrar [instrucciones detalladas en la documentación](https://ai.google.dev/gemini-api/docs/api-key).

Para hacer que la clave esté disponible a través de secretos de Kaggle, elige `Secrets` en el menú `Add-ons` y sigue las instrucciones para agregar tu clave o habilitarla para este notebook.

In [6]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

Si recibiste una respuesta de error del tipo `No user secrets exist for kernel id ...`, entonces necesitas agregar tu clave API a través de `Add-ons`, `Secrets` **y** habilitarla.

![Captura de pantalla de la casilla de verificación para habilitar el secreto GOOGLE_API_KEY](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_3.png)

### Ejecuta tu primer prompt

En este paso, probarás que tu clave API esté configurada correctamente haciendo una solicitud.

El SDK de Python utiliza un [`Client` object](https://googleapis.github.io/python-genai/genai.html#genai.client.Client) para hacer solicitudes a la API. El cliente te permite controlar qué back-end usar (entre la API de Gemini y Vertex AI) y maneja la autenticación (la clave API).

El modelo `gemini-2.0-flash` ha sido seleccionado aquí.

**Nota**: Si ves un `TransportError` en este paso, es posible que necesites **🔁 Factory reset** el notebook una vez.

In [ ]:
client = genai.Client(api_key=GOOGLE_API_KEY)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Explícame la IA como si fuera un niño.")

print(response.text)

Okay, imagine you have a really, REALLY smart robot. But instead of being made of metal and wires, this smartness lives inside a computer!

This smartness is called Artificial Intelligence, or AI for short.

Think of it like this:

*   **A puppy learning tricks:** You teach a puppy by showing it what to do and rewarding it when it does it right. AI learns in a similar way, but with lots and lots of examples and by getting "rewards" for making the right choices.
*   **A super-powered calculator:** A calculator can do math really fast. AI is like a super-powered calculator that can do lots more than just math! It can see pictures, understand words, and even play games.
*   **Playing hide-and-seek:** If you play hide-and-seek a lot, you get better at finding good hiding spots. AI is the same. The more it "plays" (or works), the better it gets at things!

**So, what can AI do?**

*   **Help you find videos you like:** Have you ever noticed that YouTube suggests videos you might enjoy? That

La respuesta a menudo viene en formato markdown, que puedes renderizar directamente en este notebook.

In [8]:
Markdown(response.text)

Okay, imagine you have a really, REALLY smart robot. But instead of being made of metal and wires, this smartness lives inside a computer!

This smartness is called Artificial Intelligence, or AI for short.

Think of it like this:

*   **A puppy learning tricks:** You teach a puppy by showing it what to do and rewarding it when it does it right. AI learns in a similar way, but with lots and lots of examples and by getting "rewards" for making the right choices.
*   **A super-powered calculator:** A calculator can do math really fast. AI is like a super-powered calculator that can do lots more than just math! It can see pictures, understand words, and even play games.
*   **Playing hide-and-seek:** If you play hide-and-seek a lot, you get better at finding good hiding spots. AI is the same. The more it "plays" (or works), the better it gets at things!

**So, what can AI do?**

*   **Help you find videos you like:** Have you ever noticed that YouTube suggests videos you might enjoy? That's AI! It's learned what kind of videos you watch and tries to guess what you'll like next.
*   **Help you spell words:** When you're typing and you misspell a word, sometimes your computer fixes it. That's AI helping you!
*   **Help doctors find diseases:** AI can look at X-rays and other medical images to help doctors find problems that might be hard to see with their own eyes.
*   **Drive cars:** Some cars are starting to drive themselves using AI! They can see the road, other cars, and even pedestrians.

**AI is still learning and growing, just like you!** It's a tool that can help us do amazing things, but it's important to remember that it's not perfect. We still need people to help it learn and make sure it's used for good.

Does that make sense? Do you have any other questions about AI?


### Inicia un chat

El ejemplo anterior utiliza una estructura de un solo turno, entrada de texto/salida de texto, pero también puedes configurar una estructura de chat de varios turnos.

In [ ]:
chat = client.chats.create(model='gemini-2.0-flash', history=[])
response = chat.send_message('¡Hola! Mi nombre es Zlork.')
print(response.text)

Nice to meet you, Zlork! It's great to have you here. How can I help you today?



In [ ]:
response = chat.send_message('¿Puedes decirme algo interesante sobre los dinosaurios?')
print(response.text)

Okay, here's something interesting about dinosaurs:

**There's evidence that some dinosaurs were brightly colored, possibly with iridescent feathers, and even different colors within the same species, similar to modern birds!**

For a long time, scientists believed dinosaurs were largely drab in color, like reptiles. However, through analyzing fossilized feathers and scales, and even the microscopic structures called melanosomes (which contain pigment), researchers have been able to infer the colors of some dinosaurs.

This has led to the discovery that some dinosaurs might have had brilliant hues, stripes, spots, and even iridescent sheens like a hummingbird. The *Microraptor*, for example, is thought to have had iridescent black feathers. And the *Sinosauropteryx* is believed to have had reddish-brown stripes.

This discovery changes our understanding of dinosaur behavior, suggesting that color may have played a role in camouflage, communication, and courtship, just like it does in m

Mientras tengas el objeto `chat` activo, el estado de la conversación
persiste. Confirma eso preguntando si recuerda el nombre del usuario.

In [ ]:
response = chat.send_message('¿Recuerdas cuál es mi nombre?')
print(response.text)

Yes, your name is Zlork.



### Elige un modelo

La API de Gemini proporciona acceso a varios modelos de la familia de modelos Gemini. Lee sobre los modelos disponibles y sus capacidades en la [página de descripción general del modelo](https://ai.google.dev/gemini-api/docs/models/gemini).

En este paso, usarás la API para listar todos los modelos disponibles.

In [12]:
for model in client.models.list():
  print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking

La respuesta de [`models.list`](https://ai.google.dev/api/models#method:-models.list) también devuelve información adicional sobre las capacidades del modelo, como los límites de tokens y los parámetros compatibles.

In [13]:
from pprint import pprint

for model in client.models.list():
  if model.name == 'models/gemini-2.0-flash':
    pprint(model.to_json_dict())
    break

{'description': 'Gemini 2.0 Flash',
 'display_name': 'Gemini 2.0 Flash',
 'input_token_limit': 1048576,
 'name': 'models/gemini-2.0-flash',
 'output_token_limit': 8192,
 'supported_actions': ['generateContent', 'countTokens'],
 'tuned_model_info': {},
 'version': '2.0'}


## Explora los parámetros de generación



### Longitud de salida

Al generar texto con un LLM, la longitud de la salida afecta el costo y el rendimiento. Generar más tokens aumenta la computación, lo que lleva a un mayor consumo de energía, latencia y costo.

Para detener el modelo de generar tokens más allá de un límite, puedes especificar el parámetro `max_output_tokens` al usar la API de Gemini. Especificar este parámetro no influye en la generación de los tokens de salida, por lo que la salida no se volverá más concisa estilística o textualmente, pero dejará de generar tokens una vez que se alcance la longitud especificada. Puede ser necesario el diseño de prompts para generar una salida más completa para tu límite dado.

In [ ]:
from google.genai import types

short_config = types.GenerateContentConfig(max_output_tokens=200)

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=short_config,
    contents='Escribe un ensayo de 1000 palabras sobre la importancia de las aceitunas en la sociedad moderna.')

print(response.text)

## The Humble Olive: A Cornerstone of Modern Society

The olive, a small, often brine-soaked fruit, may seem insignificant amidst the complexities of modern society. Yet, beneath its unassuming exterior lies a profound importance, extending far beyond its culinary applications. From its historical roots to its contributions to health, economy, and culture, the olive tree and its fruit have woven themselves into the fabric of modern life, demonstrating an enduring relevance that belies its perceived simplicity.

The story of the olive is deeply intertwined with the history of civilization. Originating in the Mediterranean basin thousands of years ago, the olive tree, *Olea europaea*, quickly became a cornerstone of ancient economies and societies. Olive oil was not merely a foodstuff; it was a source of fuel for lamps, a component of religious rituals, a base for perfumes and medicines, and a symbol of peace and prosperity. Ancient Greece valued the olive tree so highly that it was cons

In [ ]:
response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=short_config,
    contents='Escribe un poema corto sobre la importancia de las aceitunas en la sociedad moderna.')

print(response.text)

From groves sun-drenched and ancient roots,
The olive's fruit, a bounty shoots.
Pressed into oil, a golden stream,
Nourishing bodies, fulfilling dream.

On pizzas scattered, in tapenade's art,
A Mediterranean piece, a vital part.
A symbol of peace, a culinary grace,
The humble olive finds its modern place.



Explora con tus propios prompts. Prueba un prompt con un límite de salida restrictivo y luego ajusta el prompt para trabajar dentro de ese límite.

### Temperatura

La temperatura controla el grado de aleatoriedad en la selección de tokens. Las temperaturas más altas resultan en un mayor número de tokens candidatos de los cuales se selecciona el siguiente token de salida, y pueden producir resultados más diversos, mientras que las temperaturas más bajas tienen el efecto contrario, de modo que una temperatura de 0 resulta en una decodificación codiciosa, seleccionando el token más probable en cada paso.

La temperatura no proporciona ninguna garantía de aleatoriedad, pero se puede usar para "empujar" la salida de alguna manera.

In [ ]:
high_temp_config = types.GenerateContentConfig(temperature=2.0)


for _ in range(5):
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=high_temp_config,
      contents='Elige un color al azar... (responde en una sola palabra)')

  if response.text:
    print(response.text, '-' * 25)

Purple
 -------------------------
Magenta
 -------------------------
Cerulean
 -------------------------
Magenta
 -------------------------
Orange
 -------------------------


Ahora prueba el mismo prompt con la temperatura configurada a cero. Ten en cuenta que la salida no es completamente determinista, ya que otros parámetros afectan la selección de tokens, pero los resultados tienden a ser más estables.

In [ ]:
low_temp_config = types.GenerateContentConfig(temperature=0.0)

for _ in range(5):
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=low_temp_config,
      contents='Elige un color al azar... (responde en una sola palabra)')

  if response.text:
    print(response.text, '-' * 25)

Azure
 -------------------------
Azure
 -------------------------
Azure
 -------------------------
Azure
 -------------------------
Azure
 -------------------------


### Top-P

Al igual que la temperatura, el parámetro top-P también se utiliza para controlar la diversidad de la salida del modelo.

Top-P define el umbral de probabilidad que, una vez superado acumulativamente, los tokens dejan de ser seleccionados como candidatos. Un top-P de 0 es típicamente equivalente a la decodificación codiciosa, y un top-P de 1 típicamente selecciona todos los tokens en el vocabulario del modelo.

También puedes ver top-K referenciado en la literatura de LLM. Top-K no es configurable en la serie de modelos Gemini 2.0, pero se puede cambiar en modelos más antiguos. Top-K es un número entero positivo que define el número de tokens más probables de los cuales seleccionar el token de salida. Un top-K de 1 selecciona un solo token, realizando una decodificación codiciosa.


Ejecuta este ejemplo varias veces, cambia la configuración y observa el cambio en la salida.

In [ ]:
model_config = types.GenerateContentConfig(
    # Estos son los valores predeterminados para gemini-2.0-flash.
    temperature=1.0,
    top_p=0.95,
)

story_prompt = "Eres un escritor creativo. Escribe una historia corta sobre un gato que se embarca en una aventura."
response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=model_config,
    contents=story_prompt)

print(response.text)

Whiskers twitched, emerald eyes narrowed. Jasper, a ginger tabby of discerning tastes (salmon pate for breakfast, sunbeams for naps), was bored. The predictable rhythm of lapping milk and batting at dust bunnies had lost its luster. He craved… adventure.

His opportunity arrived with the morning mail. Mrs. Higgins, his beloved human, was distracted by a particularly lurid headline about a runaway llama. In the chaos, the back door swung ajar. Jasper, without a second thought, slipped through.

The world outside was a cacophony of smells and sights. Grass, ticklish and unfamiliar, carpeted the earth. Birds, unlike the predictable chirping of the cage-bound canary inside, swooped and called with wild abandon.

Jasper padded cautiously, his tail held high. He was a magnificent explorer, a fearless pioneer charting unknown territories. At least, that's what he told himself. The truth was, he was a little scared.

He soon encountered Bartholomew, a grizzled, one-eared tomcat with a permanen

## Creación de Prompts

Esta sección contiene algunos prompts del capítulo para que los pruebes directamente en la API. Intenta cambiar el texto aquí para ver cómo se desempeña cada prompt con diferentes instrucciones, más ejemplos o cualquier otro cambio que se te ocurra.

### Zero-shot

Los prompts zero-shot son prompts que describen la solicitud para el modelo directamente.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1gzKKgDHwkAvexG5Up0LMtl1-6jKMKe4g"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Abrir en AI Studio</a>
  </td>
</table>

In [ ]:
model_config = types.GenerateContentConfig(
    temperature=0.1,
    top_p=1,
    max_output_tokens=5,
)

zero_shot_prompt = """Clasifica las reseñas de películas como POSITIVAS, NEUTRALES o NEGATIVAS.
Reseña: "Her" es un estudio perturbador que revela la dirección
a la que se dirige la humanidad si se permite que la IA siga evolucionando,
sin control. Ojalá hubiera más películas como esta obra maestra.
Sentimiento: """

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=model_config,
    contents=zero_shot_prompt)

print(response.text)

POSITIVE



#### Modo Enum

Los modelos están entrenados para generar texto, y aunque los modelos Gemini 2.0 son excelentes para seguir instrucciones, otros modelos a veces pueden producir más texto del que deseas. En el ejemplo anterior, el modelo generará la etiqueta, pero a veces puede incluir una etiqueta "Sentimiento" precedente, y sin un límite de tokens de salida, también puede agregar texto explicativo después. Consulta [este prompt en AI Studio](https://aistudio.google.com/prompts/1gzKKgDHwkAvexG5Up0LMtl1-6jKMKe4g) para ver un ejemplo.

La API de Gemini tiene una función de [modo Enum](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Enum.ipynb) que te permite restringir la salida a un conjunto fijo de valores.

In [20]:
import enum

class Sentiment(enum.Enum):
    POSITIVE = "positive"
    NEUTRAL = "neutral"
    NEGATIVE = "negative"


response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        response_mime_type="text/x.enum",
        response_schema=Sentiment
    ),
    contents=zero_shot_prompt)

print(response.text)

positive


Al usar una salida restringida como un enum, el SDK de Python intentará convertir la respuesta de texto del modelo en un objeto de Python automáticamente. Se almacena en el campo `response.parsed`.

In [21]:
enum_response = response.parsed
print(enum_response)
print(type(enum_response))

Sentiment.POSITIVE
<enum 'Sentiment'>


### One-shot y few-shot

Proporcionar un ejemplo de la respuesta esperada se conoce como un prompt "one-shot". Cuando proporcionas múltiples ejemplos, es un prompt "few-shot".

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1jjWkjUSoMXmLvMJ7IzADr_GxHPJVV2bg"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Abrir en AI Studio</a>
  </td>
</table>


In [ ]:
few_shot_prompt = """Parsea el pedido de pizza de un cliente en JSON válido:

EJEMPLO:
Quiero una pizza pequeña con queso, salsa de tomate y pepperoni.
Respuesta JSON:
```
{
"size": "small",
"type": "normal",
"ingredients": ["cheese", "tomato sauce", "pepperoni"]
}
```

EJEMPLO:
¿Puedo obtener una pizza grande con salsa de tomate, albahaca y mozzarella?
Respuesta JSON:
```
{
"size": "large",
"type": "normal",
"ingredients": ["tomato sauce", "basil", "mozzarella"]
}
```

PEDIDO:
"""

customer_order = "Dame una grande con queso y piña"

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=250,
    ),
    contents=[few_shot_prompt, customer_order])

print(response.text)

```json
{
"size": "large",
"type": "normal",
"ingredients": ["cheese", "pineapple"]
}
```



#### Modo JSON

Para proporcionar control sobre el esquema y asegurarte de que solo recibas JSON (sin otro texto o markdown), puedes usar el [modo JSON](https://github.com/google-gemini/cookbook/blob/main/quickstarts/JSON_mode.ipynb) de la API de Gemini. Esto fuerza al modelo a restringir la decodificación, de modo que la selección de tokens se guíe por el esquema proporcionado.

In [ ]:
import typing_extensions as typing

class PizzaOrder(typing.TypedDict):
    size: str
    ingredients: list[str]
    type: str


response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=0.1,
        response_mime_type="application/json",
        response_schema=PizzaOrder,
    ),
    contents="¿Puedo tener una pizza de postre grande con manzana y chocolate?")

print(response.text)

{
  "size": "large",
  "ingredients": ["apple", "chocolate"],
  "type": "dessert pizza"
}


### Cadena de Pensamiento (CoT)

El prompting directo en LLMs puede devolver respuestas rápidamente y (en términos de uso de tokens de salida) de manera eficiente, pero pueden ser propensos a la alucinación. La respuesta puede "parecer" correcta (en términos de lenguaje y sintaxis) pero es incorrecta en términos de veracidad y razonamiento.

El prompting de Cadena de Pensamiento es una técnica en la que instruyes al modelo para que genere pasos de razonamiento intermedios, y típicamente obtiene mejores resultados, especialmente cuando se combina con ejemplos few-shot. Vale la pena señalar que esta técnica no elimina completamente las alucinaciones, y que tiende a costar más ejecutarla, debido al aumento en el conteo de tokens.

Modelos como la familia Gemini están entrenados para ser "conversadores" o "reflexivos" y proporcionarán pasos de razonamiento sin prompting, por lo que para este simple ejemplo puedes pedirle al modelo que sea más directo en el prompt para forzar una respuesta sin razonamiento. Intenta volver a ejecutar este paso si el modelo tiene suerte y obtiene la respuesta correcta en el primer intento.

In [ ]:
prompt = """Cuando tenía 4 años, mi pareja tenía 3 veces mi edad. Ahora, tengo 20 años. ¿Cuántos años tiene mi pareja? Devuelve la respuesta directamente."""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt)

print(response.text)

52



Ahora intenta el mismo enfoque, pero indica al modelo que "piense paso a paso".

In [ ]:
prompt = """Cuando tenía 4 años, mi pareja tenía 3 veces mi edad. Ahora,
tengo 20 años. ¿Cuántos años tiene mi pareja? Pensemos paso a paso."""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt)

Markdown(response.text)

Here's how to solve this:

1. **Find the age difference:** When you were 4, your partner was 3 times your age, meaning they were 4 * 3 = 12 years old.

2. **Calculate the age difference:** The age difference between you and your partner is 12 - 4 = 8 years.

3. **Determine partner's current age:** Since the age difference remains constant, your partner is currently 20 + 8 = 28 years old.

**Answer:** Your partner is currently 28 years old.


### ReAct: Razonar y actuar

En este ejemplo, ejecutarás un prompt ReAct directamente en la API de Gemini y realizarás los pasos de búsqueda tú mismo. Como este prompt sigue una estructura bien definida, existen marcos disponibles que envuelven el prompt en APIs más fáciles de usar que realizan llamadas a herramientas automáticamente, como el ejemplo de LangChain del documento técnico sobre "Prompting".

Para probar esto con el motor de búsqueda de Wikipedia, consulta el [ejemplo de búsqueda en Wikipedia con ReAct](https://github.com/google-gemini/cookbook/blob/main/examples/Search_Wikipedia_using_ReAct.ipynb) en el libro de recetas.


> Nota: El prompt y los ejemplos en contexto utilizados aquí son de [https://github.com/ysymyth/ReAct](https://github.com/ysymyth/ReAct) que se publica bajo una [licencia MIT](https://opensource.org/licenses/MIT), Copyright (c) 2023 Shunyu Yao.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/18oo63Lwosd-bQ6Ay51uGogB3Wk3H8XMO"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Abrir en AI Studio</a>
  </td>
</table>


In [ ]:
model_instructions = """
Resuelve una tarea de respuesta a preguntas con pasos intercalados de Pensamiento, Acción y Observación. El Pensamiento puede razonar sobre la situación actual,
la Observación es comprender información relevante de la salida de una Acción y la Acción puede ser de tres tipos:
 (1) <search>entidad</search>, que busca la entidad exacta en Wikipedia y devuelve el primer párrafo si existe. Si no,
     devolverá algunas entidades similares para buscar y puedes intentar buscar la información de esos temas.
 (2) <lookup>palabra clave</lookup>, que devuelve la siguiente oración que contiene la palabra clave en el contexto actual. Esto solo hace coincidencias exactas,
     así que mantén tus búsquedas cortas.
 (3) <finish>respuesta</finish>, que devuelve la respuesta y finaliza la tarea.
"""

example1 = """Pregunta
El músico y satirista Allie Goertz escribió una canción sobre el personaje de "Los Simpson" Milhouse, ¿a quién nombró Matt Groening?

Pensamiento 1
La pregunta se simplifica a "El personaje de Los Simpson Milhouse es nombrado después de quién". Solo necesito buscar Milhouse y encontrar a quién está nombrado.

Acción 1
<search>Milhouse</search>

Observación 1
Milhouse Mussolini Van Houten es un personaje recurrente en la serie de televisión animada de Fox Los Simpson, con la voz de Pamela Hayden y creado por Matt Groening.

Pensamiento 2
El párrafo no dice a quién está nombrado Milhouse, tal vez pueda buscar "nombrado después de".

Acción 2
<lookup>nombrado después de</lookup>

Observación 2
Milhouse fue nombrado después del presidente de EE. UU. Richard Nixon, cuyo segundo nombre era Milhous.

Pensamiento 3
Milhouse fue nombrado después del presidente de EE. UU. Richard Nixon, así que la respuesta es Richard Nixon.

Acción 3
<finish>Richard Nixon</finish>
"""

example2 = """Pregunta
¿Cuál es el rango de elevación para el área en la que se extiende el sector oriental de la orogenia de Colorado?

Pensamiento 1
Necesito buscar la orogenia de Colorado, encontrar el área en la que se extiende el sector oriental de la orogenia de Colorado, luego encontrar el rango de elevación del área.

Acción 1
<search>orogenia de Colorado</search>

Observación 1
La orogenia de Colorado fue un episodio de construcción de montañas (una orogenia) en Colorado y áreas circundantes.

Pensamiento 2
No menciona el sector oriental. Así que necesito buscar el sector oriental.

Acción 2
<lookup>sector oriental</lookup>

Observación 2
El sector oriental se extiende hacia las Grandes Llanuras y se llama la orogenia de las Llanuras Centrales.

Pensamiento 3
El sector oriental de la orogenia de Colorado se extiende hacia las Grandes Llanuras. Así que necesito buscar las Grandes Llanuras y encontrar su rango de elevación.

Acción 3
<search>Grandes Llanuras</search>

Observación 3
Las Grandes Llanuras se refieren a una de dos regiones terrestres distintas

Pensamiento 4
Necesito buscar en su lugar las Grandes Llanuras (Estados Unidos).

Acción 4
<search>Grandes Llanuras (Estados Unidos)</search>

Observación 4
Las Grandes Llanuras son una subregión de las Grandes Llanuras. De este a oeste, las Grandes Llanuras se elevan en elevación desde alrededor de 1,800 a 7,000 pies (550 a 2,130m).

Pensamiento 5
Las Grandes Llanuras se elevan en elevación desde alrededor de 1,800 a 7,000 pies, así que la respuesta es 1,800 a 7,000 pies.

Acción 5
<finish>1,800 a 7,000 pies</finish>
"""

# Inventa más ejemplos tú mismo, o echa un vistazo a https://github.com/ysymyth/ReAct/

Para capturar un solo paso a la vez, mientras ignoras cualquier paso de Observación alucinado, usarás `stop_sequences` para finalizar el proceso de generación. Los pasos son `Pensamiento`, `Acción`, `Observación`, en ese orden.

In [ ]:
question = """Pregunta
¿Quién fue el autor más joven listado en el artículo de transformers NLP?
"""

# Realizarás la Acción; así que genera hasta, pero no incluyendo, la Observación.
react_config = types.GenerateContentConfig(
    stop_sequences=["\nObservación"],
    system_instruction=model_instructions + example1 + example2,
)

# Crea un chat que tenga las instrucciones del modelo y los ejemplos pre-cargados.
react_chat = client.chats.create(
    model='gemini-2.0-flash',
    config=react_config,
)

resp = react_chat.send_message(question)
print(resp.text)

Thought 1
I need to find the transformers NLP paper and look for the author information. Then find the youngest author.

Action 1
<search>transformers NLP paper</search>



Ahora puedes realizar esta investigación tú mismo y proporcionarla de nuevo al modelo.

In [ ]:
observation = """Observación 1
[1706.03762] Attention Is All You Need
Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Proponemos una nueva arquitectura de red simple, el Transformer, basada únicamente en mecanismos de atención, prescindiendo completamente de la recurrencia y las convoluciones.
"""
resp = react_chat.send_message(observation)
print(resp.text)

Thought 2
I have found the paper, now I need to find the age of each author and find the youngest. This is difficult without searching each author individually. I will try to search for the ages of the authors.

Action 2
<search>Ashish Vaswani age</search>



Este proceso se repite hasta que se alcanza la acción `<finish>`. Puedes continuar ejecutando esto tú mismo si lo deseas, o probar el [ejemplo de Wikipedia](https://github.com/google-gemini/cookbook/blob/main/examples/Search_Wikipedia_using_ReAct.ipynb) para ver un sistema ReAct completamente automatizado en acción.

## Modo de pensamiento

El modelo experimental Gemini Flash 2.0 "Thinking" ha sido entrenado para generar el "proceso de pensamiento" que el modelo atraviesa como parte de su respuesta. Como resultado, el modelo Flash Thinking es capaz de proporcionar respuestas con capacidades de razonamiento más fuertes.

Usar un modelo de "modo de pensamiento" puede proporcionarte respuestas de alta calidad sin necesidad de prompting especializado como los enfoques anteriores. Una razón por la que esta técnica es efectiva es que induces al modelo a generar información relevante ("lluvia de ideas" o "pensamientos") que luego se utiliza como parte del contexto en el que se genera la respuesta final.

Ten en cuenta que cuando usas la API, obtienes la respuesta final del modelo, pero los pensamientos no se capturan. Para ver los pensamientos intermedios, prueba el [modelo de modo de pensamiento en AI Studio](https://aistudio.google.com/prompts/new_chat?model=gemini-2.0-flash-thinking-exp-01-21).

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1Z991SV7lZZZqioOiqIUPv9a9ix-ws4zk"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Abrir en AI Studio</a>
  </td>
</table>

In [ ]:
import io
from IPython.display import Markdown, clear_output


response = client.models.generate_content_stream(
    model='gemini-2.0-flash-thinking-exp',
    contents='¿Quién fue el autor más joven listado en el artículo de transformers NLP?',
)

buf = io.StringIO()
for chunk in response:
    buf.write(chunk.text)
    # Muestra la respuesta a medida que se transmite
    print(chunk.text, end='')

# Y luego renderiza la respuesta final como markdown formateado.
clear_output()
Markdown(buf.getvalue())

Based on the information available online, and considering typical academic career paths, **Aidan N. Gomez** appears to be the youngest author listed on the "Attention is All You Need" paper, which introduced the Transformer architecture.

Here's why:

* **Aidan N. Gomez** was an undergraduate student at the University of Oxford at the time the paper was written.  His LinkedIn profile indicates he graduated from Oxford in 2017, the same year the paper was published.  This typically places someone in their early 20s when publishing such a paper as an undergraduate.

* The other authors were researchers at Google Brain, University of Toronto, and RWTH Aachen University, and many held PhDs or were in advanced stages of their research careers. This generally suggests they were older than an undergraduate student.

While precise birthdates aren't readily available for all authors to definitively confirm age,  the academic stage of Aidan N. Gomez at the time of publication strongly suggests he was the youngest author on the paper.

Therefore, the most likely answer is **Aidan N. Gomez**.

## Creación de código

### Generación de código

La familia de modelos Gemini se puede utilizar para generar código, configuraciones y scripts. Generar código puede ser útil al aprender a programar, aprender un nuevo lenguaje o para generar rápidamente un primer borrador.

Es importante tener en cuenta que, dado que los LLMs pueden cometer errores y pueden repetir datos de entrenamiento, es esencial leer y probar tu código primero, y cumplir con cualquier licencia relevante.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1YX71JGtzDjXQkgdes8bP6i3oH5lCRKxv"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Abrir en AI Studio</a>
  </td>
</table>

In [ ]:
# A los modelos Gemini les encanta hablar, así que ayuda especificar que se ciñan al código si eso
# es todo lo que deseas.
code_prompt = """
Escribe una función en Python para calcular el factorial de un número. Sin explicación, proporciona solo el código.
"""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=1,
        top_p=1,
        max_output_tokens=1024,
    ),
    contents=code_prompt)

Markdown(response.text)

```python
def factorial(n):
  """Calculate the factorial of a number."""
  if n == 0:
    return 1
  else:
    return n * factorial(n-1)
```


### Ejecución de código

La API de Gemini también puede ejecutar automáticamente el código generado y devolverá la salida.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/11veFr_VYEwBWcLkhNLr-maCG0G8sS_7Z"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Abrir en AI Studio</a>
  </td>
</table>

In [ ]:
from pprint import pprint

config = types.GenerateContentConfig(
    tools=[types.Tool(code_execution=types.ToolCodeExecution())],
)

code_exec_prompt = """
Genera los primeros 14 números primos impares, luego calcula su suma.
"""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=config,
    contents=code_exec_prompt)

for part in response.candidates[0].content.parts:
  pprint(part.to_json_dict())
  print("-----")

{'text': 'Okay, I can do that. First, I will generate the first 14 odd prime '
         'numbers. Remember that a prime number is a number greater than 1 '
         'that has only two divisors: 1 and itself. Also, note that 2 is the '
         'only even prime number, so all other prime numbers are odd. After '
         'generating these numbers, I will sum them.\n'
         '\n'}
-----
{'executable_code': {'code': 'def is_prime(n):\n'
                             '    if n <= 1:\n'
                             '        return False\n'
                             '    if n <= 3:\n'
                             '        return True\n'
                             '    if n % 2 == 0 or n % 3 == 0:\n'
                             '        return False\n'
                             '    i = 5\n'
                             '    while i * i <= n:\n'
                             '        if n % i == 0 or n % (i + 2) == 0:\n'
                             '            return False\n'
     

Esta respuesta contiene múltiples partes, incluyendo una parte de texto de apertura y cierre que representan respuestas regulares, una parte `executable_code` que representa el código generado y una parte `code_execution_result` que representa los resultados de ejecutar el código generado.

Puedes explorarlas individualmente.

In [ ]:
for part in response.candidates[0].content.parts:
    if part.text:
        display(Markdown(part.text))
    elif part.executable_code:
        display(Markdown(f'```python\n{part.executable_code.code}\n```'))
    elif part.code_execution_result:
        if part.code_execution_result.outcome != 'OUTCOME_OK':
            display(Markdown(f'## Estado {part.code_execution_result.outcome}'))

        display(Markdown(f'```\n{part.code_execution_result.output}\n```'))

Okay, I can do that. First, I will generate the first 14 odd prime numbers. Remember that a prime number is a number greater than 1 that has only two divisors: 1 and itself. Also, note that 2 is the only even prime number, so all other prime numbers are odd. After generating these numbers, I will sum them.



```python
def is_prime(n):
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

primes = []
num = 3
while len(primes) < 14:
    if is_prime(num):
        primes.append(num)
    num += 2

print(f'{primes=}')

sum_of_primes = sum(primes)
print(f'{sum_of_primes=}')

```

```
primes=[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
sum_of_primes=326

```

The first 14 odd prime numbers are 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, and 47.

The sum of these prime numbers is 326.


### Explicación de código

La familia de modelos Gemini también puede explicarte el código. En este ejemplo, pasas un [script de bash](https://github.com/magicmonty/bash-git-prompt) y haces algunas preguntas.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1N7LGzWzCYieyOf_7bAG4plrmkpDNmUyb"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Abrir en AI Studio</a>
  </td>
</table>

In [ ]:
file_contents = !curl https://raw.githubusercontent.com/magicmonty/bash-git-prompt/refs/heads/master/gitprompt.sh

explain_prompt = f"""
Por favor, explica qué hace este archivo a un nivel muy alto. ¿Qué es y por qué lo usaría?

```
{file_contents}
```
"""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=explain_prompt)

Markdown(response.text)

This file, `git-prompt.sh`, is a shell script designed to enhance your command-line prompt when working with Git repositories.  It provides information about the current Git status directly in your prompt, such as the branch name, the state of tracked/untracked files, and the divergence from the remote repository.

**In essence, it's a cosmetic and productivity tool for developers using Git.**

Here's a breakdown of why you might use it:

*   **Git Status at a Glance:**  Instead of constantly running `git status`, the script displays key information (branch, modified files, etc.) directly in your prompt.
*   **Improved Workflow:**  Knowing the Git status immediately helps you avoid mistakes and stay organized.
*   **Customization:**  The script offers many options for customizing the prompt's appearance, including colors, symbols, and the information displayed.
*   **Virtual Environment awareness**: The script also shows any virtual environments that you are working on, such as virtualenv, node virtualenv, and conda environments.

In short, this file helps you create a more informative and visually appealing command-line prompt that integrates seamlessly with Git.


## Aprende más

Para aprender más sobre la creación de prompts en profundidad:

* Consulta el documento técnico emitido con el contenido de hoy,
* Prueba las aplicaciones listadas en la parte superior de este notebook ([TextFX](https://textfx.withgoogle.com/), [SQL Talk](https://sql-talk-r5gdynozbq-uc.a.run.app/) y [NotebookLM](https://notebooklm.google/)),
* Lee la [Introducción a la creación de prompts](https://ai.google.dev/gemini-api/docs/prompting-intro) de la documentación de la API de Gemini,
* Explora la [galería de prompts](https://ai.google.dev/gemini-api/prompts) de la API de Gemini y pruébalos en AI Studio,
* Consulta el libro de recetas de la API de Gemini para [ejemplos inspiradores](https://github.com/google-gemini/cookbook/blob/main/examples/) y [inicios rápidos educativos](https://github.com/google-gemini/cookbook/blob/main/quickstarts/).

Asegúrate de revisar los laboratorios de código del día 3 también, donde explorarás una creación de prompts más avanzada con ejecución de código.

¡Y por favor comparte cualquier cosa emocionante que hayas probado en el Discord!

*- [Mark McD](https://linktr.ee/markmcd)*